In [3]:
from PIL import Image
import numpy as np
from os import path
import cv2

def image_to_np_array(image_name: str) -> np.array:
    img_src = Image.open(path.join(image_name)).convert('RGB')
    return np.array(img_src)

def two_step_resampling(img: np.array, numerator: int, denominator: int) -> np.array:
    tmp = one_step_resampling(img, numerator, lambda a, b: a * b, lambda a, b: int(round(a / b)))
    return one_step_resampling(tmp, denominator, lambda a, b: int(round(a / b)), lambda a, b: a * b)

def one_step_resampling(img: np.array, factor: float, f1, f2):
    dimensions = img.shape[0:2]
    new_dimensions = tuple(f1(dimension, factor) for dimension in dimensions)
    new_shape = (*new_dimensions, img.shape[2])
    new_img = np.empty(new_shape)

    for x in range(new_dimensions[0]):
        for y in range(new_dimensions[1]):
            new_img[x, y] = img[min(f2(x, factor), dimensions[0] - 1), min(f2(y, factor), dimensions[1] - 1)]
    return new_img

def safe_number_input(number_type: type, lower_bound=None, upper_bound=None):
    input_correct = False
    user_input = 0

    while not input_correct:
        try:
            user_input = number_type(input('> '))
            if lower_bound is not None and user_input < lower_bound:
                raise ValueError
            if upper_bound is not None and user_input > upper_bound:
                raise ValueError
            input_correct = True
        except ValueError:
            print("Введите корректное значение")
    return user_input

def execute(img, f1, f2, number_type=int):
    data_type = np.uint8
    color_model = 'RGB'

    factor = safe_number_input(number_type, 0.5)
    result = Image.fromarray(one_step_resampling(img, factor, f1, f2).astype(data_type), color_model)

    return result


if __name__ == '__main__':
    img = image_to_np_array('img.png')

    print('Выберите операцию: 1)Интерполяция 2)Децимация 3)Двухпроходная передискретизация 4)Однопроходная передискретизация')
    enter = int(input('> '))
    

    match enter:
        case 1:
            print('Введите целый коэффициент растяжения')
            result = execute(img, lambda a, b: a * b, lambda a, b: int(round(a / b)))

        case 2:
            print('Введите целый коэффициент сжатия')
            result = execute(img, lambda a, b: int(round(a / b)), lambda a, b: a * b)

        case 3:
            print('Введите целый коэффициент растяжения')
            numerator = safe_number_input(int, 1)

            print('Введите целый коэффициент сжатия')
            denominator = safe_number_input(int, 1)

            args = [numerator, denominator]
            result = Image.fromarray(two_step_resampling(img, *args).astype(np.uint8), 'RGB')

        case 4:
            print('Введите дробный коэффициент растяжения/сжатия')
            result = execute(img, lambda a, b: int(round(a * b)), lambda a, b: int(round(a / b)), float)

        case _:
            exit()



    result_array = np.array(result)
    cv2.imshow('Orig', img)
    cv2.imshow('Image', result_array)
    cv2.waitKey(0)  
    cv2.destroyAllWindows()

Выберите операцию: 1)Интерполяция 2)Децимация 3)Двухпроходная передискретизация 4)Однопроходная передискретизация


>  1


Введите целый коэффициент растяжения


>  2
